In [1]:
import sys
import os
sys.path.append("../")
sys.path.append("../..")

from KETIPrePartialDataPreprocessing import makeNaNImputationTest

inputType ='file' # or file
refine_param = {'removeDuplication':True, 'staticFrequency':True}
outlier_param= {'certainOutlierToNaN':True, 'uncertainOutlierToNaN':True, 'data_type':'air'}
imputation_param ={
"imputation_method":[
    {"min":0,"max":2,"method":"linear"}],"totalNanLimit":90}

# mean, median X
# ffill~polynomial_interpolate=> 각 구간마다 딱 고만큼만 Fill

input_data = makeNaNImputationTest.inputControl(inputType)
from KETIPrePartialDataPreprocessing import data_preprocessing
import numpy as np
input_test_data = input_data.replace(4.0, np.nan)

In [2]:
from KETIPrePartialDataPreprocessing import data_preprocessing

MDP = data_preprocessing.DataPreprocessing()
###########
print("=============")
print('original', input_test_data.isna().sum())
print(input_data[:20])
refined_data = MDP.get_refinedData(input_data, refine_param)
print("=============")
print('after refine', refined_data.isna().sum())
print(refined_data[:20])
###########
datawithMoreCertainNaN, datawithMoreUnCertainNaN = MDP.get_outlierToNaNData(refined_data, outlier_param)
print("=============")
print('after outlierDetection', datawithMoreUnCertainNaN.isna().sum())
print(datawithMoreUnCertainNaN[:20])
###########

### TODO ST Oh
imputed_data = MDP.get_imputedData(datawithMoreUnCertainNaN, imputation_param)
print("=============")
print('after imputation', imputed_data.isna().sum())
print(imputed_data[:20])
    
from KETIPrePartialDataPreprocessing.dataRemovebyNaN import data_remove_byNaN
nanM = data_remove_byNaN.DataRemoveByNaNStatus()
nanMap = nanM.consecutiveNaNCountMap(datawithMoreUnCertainNaN)

original temp    17428
co2     12820
pm10        0
dtype: int64
                     temp  co2    pm10
timedate                              
2021-01-28 12:51:00   3.0  5.0 -9999.0
2021-01-28 12:52:00   3.0  5.0    64.0
2021-01-28 12:53:00   4.0  6.0    56.0
2021-01-28 12:54:00   4.0  7.0    59.0
2021-01-28 12:55:00   4.0  7.0    59.0
2021-01-28 12:56:00   3.0  6.0    58.0
2021-01-28 12:57:00   4.0  7.0    57.0
2021-01-28 12:58:00   4.0  7.0    58.0
2021-01-28 12:59:00   4.0  6.0    57.0
2021-01-28 13:00:00   4.0  7.0    56.0
2021-01-28 13:01:00   4.0  6.0    58.0
2021-01-28 13:02:00   4.0  7.0    57.0
2021-01-28 13:03:00   4.0  6.0    56.0
2021-01-28 13:04:00   4.0  6.0    57.0
2021-01-28 13:05:00   3.0  6.0    59.0
2021-01-28 13:06:00   3.0  6.0    61.0
2021-01-28 13:07:00   3.0  6.0    59.0
2021-01-28 13:08:00   3.0  6.0    61.0
2021-01-28 13:09:00   3.0  5.0    60.0
2021-01-28 13:10:00   3.0  6.0    60.0
after refine temp    212635
co2     212635
pm10    212635
dtype: int64
       

In [4]:
nanMap

,temp,co2,pm10
timedate,,,
2021-01-28 12:51:00,0,0,1
2021-01-28 12:52:00,0,0,0
2021-01-28 12:53:00,0,0,0
2021-01-28 12:54:00,0,0,0
2021-01-28 12:55:00,1,1,1
...,...,...,...
2021-07-01 17:59:00,12,12,12
2021-07-01 18:00:00,13,13,13
2021-07-01 18:01:00,14,14,14
